***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
# !gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
idoo-cluster  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import numpy as np
from pyspark.sql import Row
from pyspark.sql import SQLContext


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
# !ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 10 11:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [9]:
# spark

In [23]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'wikii_204767651' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [ ]:
def save_index_to_pickle(name, object_to_save):
  with open(f"{name}.pkl", "wb") as f:
      pickle.dump(object_to_save, f)

  client = storage.Client()
  bucket = client.bucket(bucket_name)
  blob_posting_locs = bucket.blob(f"postings_gcp/{name}.pkl")
  blob_posting_locs.upload_from_filename(f"{name}.pkl")
    

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [24]:
parquetFile = spark.read.parquet(*paths)

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [25]:
# Count number of wiki pages
# parquetFile.count()

6348910

In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [26]:
from inverted_index_gcp import InvertedIndex

## **All functions to build Inverted Index**


In [27]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id, reverse = False):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  freq_counter = Counter();
  tokens = [token for token in tokens if token not in all_stopwords]

  freq_counter.update(tokens)

  if reverse == True:
      return [(id, (token, freq_counter[token])) for token in freq_counter]

  return [(token, (id, freq_counter[token])) for token in freq_counter]



def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  sorted_by_first = sorted(unsorted_pl, key=lambda tup: tup[0])


  return sorted_by_first


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  return postings.mapValues(lambda x: len(x))

def calculate_term(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  return postings.mapValues(lambda x: len(x))

def partition_postings_and_write(postings, file_name=""):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  return postings.map(lambda element: (token2bucket_id(element[0]), [element])).reduceByKey(lambda x,y: x+y).map(lambda posting: InvertedIndex.write_a_posting_list(posting, bucket_name, file_name=file_name))


Get the query train dict

In [28]:
train_dict = {"best marvel movie": [60283633, 61073786, 5676692, 56289553, 60774345, 27306717, 61592102, 42163310, 60952488, 36439749, 48530084, 10589717, 29129051, 59892, 612052, 44254295, 878659, 54653881, 51430647, 66111204, 22114132, 55935213, 41677925, 17296107, 61651800, 9110929, 67055, 37497391, 60744481, 65464184, 41974496, 60616450, 60463979, 65967176, 57069491, 46208997, 22144990, 62372638, 1074657, 44240443, 33463661, 41974555, 43603241, 33700618, 5027882, 66423851, 60754840], "How do kids come to world?": [15474, 1357127, 636806, 43033258, 6271835, 56480301, 23133297, 615418, 73165, 24470328, 1833777, 1380383, 79449, 4827661, 387703, 18863597, 36827305, 494299, 194687, 5591344, 48490547, 634139, 42072639, 44311171, 29384326, 1908019, 296627, 11263877, 101942, 2045465, 56921904, 128987, 22888933, 1072968, 25490788, 83449, 884998, 1151454, 30640885, 35072597, 2535885, 30861, 51046955, 13603, 3060346, 88380, 19698110, 72214, 6236554, 46105], "Information retrieval": [1897206, 10179411, 25130414, 5818361, 1185840, 20948989, 48317971, 509628, 494528, 11486091, 50716473, 24963841, 296950, 35804330, 261193, 15271, 39000674, 19988623, 38156944, 36794719, 731640, 14109784, 10328235, 25935906, 16635934, 33407925, 743971, 3781784, 14343887, 57312392, 24997830, 442684, 7872152, 14473878, 25959000, 9511414], "LinkedIn": [3591502, 55679006, 970755, 36070366, 63641225, 41726116, 51562019, 35549457, 21179478, 62976368, 27769500, 57147095, 31403505, 22291643, 50191962], "How to make coffee?": [4506407, 321546, 37249793, 17668101, 26731675, 6887661, 1566948, 5612891, 211895, 68117784, 4604645, 47660, 3757402, 273707, 8866584, 5964683, 49099835, 28890200, 53151326, 300805, 1623162, 3775558, 273700, 667037, 5212064, 6826364, 63534797, 54459918, 604727, 30860428, 2461806, 3639440, 2929216, 12343966, 408360, 63520964, 838057, 6332026, 19619306, 215424, 482824, 38579961, 8728856, 2165666, 3785715, 366244, 1646753, 31824340], "Ritalin": [649100, 8802530, 6428730, 608718, 13594085, 66391, 25164479, 24754461, 22611786, 964614, 7432624, 5721484, 57068567, 1333695, 4387617, 463961, 23891416, 56961277, 47956615, 4726434, 52780757, 50762105, 40542151, 1186041, 10671710, 7594242, 57762, 2580091, 159284, 2495940, 6281833, 45690249, 1546447, 32325617, 205878, 1790029, 5497377], "How to make wine at home?": [373172, 3602925, 20790067, 223834, 15468138, 3398365, 61014433, 19600890, 927688, 146918, 22216378, 1417287, 13824744, 57098, 3276784, 466664, 41337483, 1031040, 36029170, 29324283, 26924822, 31505523, 13532634, 4378282, 1045027, 1455948, 14825456, 485220, 37468361, 1041458, 8177057, 2866516, 31704630, 21991369, 4554556, 713636, 8608425, 20810258, 22777652, 1039412, 32961, 8778890, 683094, 19561784, 6032951, 10998, 5222577, 7414829, 20185928, 8318345], "Most expensive city in the world": [33508970, 3602421, 94167, 24724090, 30057, 220886, 31453, 19058, 31326350, 32706, 645042, 3928523, 18402, 34374079, 522934, 13476079, 2376810, 36511, 172538, 15218891, 390875, 22309, 12521, 65708464, 1664254, 35368654, 19004, 309890, 27862, 27318, 45470, 10992, 53446, 19261, 19189, 3848717, 11947794, 49749249, 7780, 14900757, 9299090, 26976, 49728, 63946361, 302201], "India": [141896, 14745, 24452, 265059, 14597, 13890, 42737, 2377, 1186115, 6825785, 26457880, 1472206, 17359901, 37756, 53707, 315776, 4208015, 295335, 14598, 1996872, 764545, 1108803, 3574003, 678583, 7564733, 37534, 2198463, 720414, 6622547, 1683930, 231623, 17774253, 14533, 19189, 275047, 20611562, 43281, 17719886, 10710364, 5864614, 3315459, 14580, 47905, 3799826, 553883, 375986, 408215], "how to make money fast?": [67987778, 12789839, 5624681, 44379765, 400777, 47720307, 45332, 1531043, 48732, 7322279, 51895777, 65228, 60739751, 21175589, 846772, 9833167, 22226313, 63809606, 35666788, 1527716, 4416646, 23830729, 264058, 32595633, 1335238, 12020461, 1793651, 1370831, 63121, 2913859, 42994, 4090453, 17418777, 5145001, 43250171, 8957449, 43030666, 473309, 624998, 7555986, 22156522, 13681, 29681566, 17362858, 19390, 407288, 1276547, 2763667], "Netflix": [65595607, 34075129, 50602056, 65741484, 32670973, 61972257, 66174045, 47048067, 49016960, 63732884, 175537, 56312051, 65073808, 59629338, 54671372, 56312054, 50276542, 57041239, 66422422, 67450679, 66299065, 9399111, 50137861, 40030145], "Apple computer": [254496, 50865995, 5285468, 5653238, 3356874, 345676, 2275, 4478297, 2593693, 3608414, 18640, 248101, 15183570, 20647724, 1159939, 17826747, 619983, 856, 46728817, 2116, 1492625, 77118, 32327247, 15357987, 400593, 17997437, 1005263, 345354, 2020710, 660310, 1344, 19006979, 15295713, 2786155, 2117, 21694, 233780, 5078775, 73262, 21347643, 27848, 548115], "The Simpsons": [19293758, 1424178, 74813, 1625137, 34519668, 4939408, 11028525, 49387265, 4939471, 292279, 60534017, 9306179, 33350134, 4939519, 1466966, 4939306, 4939444, 140332, 4939501, 29838, 5451605, 19266557, 3038969, 14040227, 4939334, 188572, 10765975, 22423628, 4776930], "World cup": [32516422, 42931572, 2996777, 33727, 183628, 60637832, 8821389, 16842834, 22230053, 1166428, 29868391, 64467696, 4743361, 13327177, 61269058, 26814387, 62528055, 10822574, 3482503, 36581929, 8258172, 16966712, 39302261, 244862, 67608822, 1853149, 39812824, 55490096, 2150801, 8734046, 32352129, 16383, 59707, 19537336, 3556431, 17742072, 11370, 656933, 168079, 41648358, 4723188, 1248592], "How to lose weight?": [400199, 1151047, 791546, 67730903, 27300359, 84252, 26639763, 8581665, 1148926, 64543917, 6319249, 2029766, 56885915, 11665493, 1958879, 28396636, 56435, 2883760, 31429041, 32051848, 277790, 11884255, 49051658, 1017976, 42528947, 1149933, 65004286, 4748844, 44442017, 35281209, 40925771, 30687447, 11249433, 45280337, 17659030, 8460, 3549164, 727293, 28541957, 12523816, 33825347, 18168862, 9972157, 410007, 27148738], "Java": [1179384, 17521476, 5516020, 5863400, 15628, 4093054, 135063, 663788, 9845, 1455590, 3901428, 731735, 1079500, 24920873, 11125049, 7955681, 38321273, 456722, 15881, 16389, 26257672, 43284, 651278, 127604, 43826, 314356, 53078721, 611589, 1131136, 230828, 417018, 42870, 69336, 4718446, 1414212, 7811267, 42871, 40659966, 13593, 1326984, 453584, 320443, 30120784, 7771171, 269441, 4294832], "Air Jordan": [3647739, 3890370, 6722408, 105344, 18998781, 1371219, 60601430, 7851893, 28155315, 1394509, 4253801, 36916362, 265033, 23353937, 13365219, 20455, 3097723, 50066979, 51546226, 2310146, 67838974, 9998569, 62741501, 58209447], "how to deal with depression?": [2721889, 13190302, 63499429, 16360289, 39218436, 33310173, 2367697, 57688, 20529621, 4041101, 49233423, 2685269, 840273, 25258288, 43600438, 60611538, 19283335, 18550003, 33255495, 19356, 60457349, 2891701, 66811, 34753948, 43875835, 42730418, 717119, 1295947, 18176448, 2353519, 1879108, 14325087, 3440273, 175357, 16407460, 3762294, 4531, 19064282, 52316, 8389, 255475, 341658, 20448627, 22481627, 21211994, 5144613, 30846934, 1500618, 234796], "How do you make gold": [323246, 5580137, 1686492, 1385632, 23290471, 6890967, 15739, 39740796, 62929, 1020809, 251087, 6109962, 6996576, 402244, 2015573, 20063724, 1230653, 180211, 7133952, 23324, 12240, 1291393, 3519942, 12095348, 44712684, 27119, 886856, 18300514, 25918508, 37412, 2526649, 39639653, 390698, 1356272, 10865561, 1386629, 5024105, 3706246, 67110306, 2732267, 15457257, 56226, 19074264, 63280480, 1581831, 45756, 2927992, 27345986, 152176], "Marijuana": [60920, 52227830, 22707918, 4512923, 68188835, 28985374, 31188467, 52184272, 52209782, 27202445, 20481920, 1481886, 19920359, 2331004, 19357, 44975261, 145891, 28572685, 20566488, 37646421, 383537, 20866399, 53836251, 150113, 53871120, 19760623, 3045683, 8596369, 1227367, 168917, 14942276, 48640150, 52342272, 52356241, 56078060, 38310, 175440, 53897655, 52228042, 52183794, 11164587, 168915, 48920848, 47227709, 23154203, 184488], "How to make hummus": [2322115, 9513043, 3260137, 7329519, 3736012, 8559295, 164311, 13607, 682549, 7489122, 289691, 24230253, 22736969, 49643204, 52682605, 3841447, 3099917, 47863605, 5033181, 1626287, 64051004, 42947658, 11287682, 11447140, 23619350, 5334377, 2243880, 1039663, 53350936, 3508935, 62166289, 20657443, 56494240, 4925720, 38936168, 82789, 48876576, 75065, 607255, 11577897, 453166, 57146, 14320, 2578570, 3548013], "Winter": [1511596, 52709838, 1673945, 17349106, 8521120, 3060382, 16615604, 8438818, 8351234, 979072, 64928991, 36439749, 1817908, 1971153, 30276826, 4538366, 1298502, 22933429, 6511088, 38950, 19431459, 1221144, 19938267, 1843684, 34069, 6201653, 33672235, 3548574, 1372169, 65601132, 38416091, 1088531, 2020857, 316711, 34061, 9825536, 22190045, 43343961, 1632099, 962053, 961505], "Rick and Morty": [54046846, 49170369, 55339286, 62417830, 54251265, 41283158, 65819511, 49029294, 67520032, 49134382, 42311608, 63656330, 52261594, 41185040, 43794572, 64413225, 49131135, 49127974, 43794574, 63656361, 49128142, 41699729, 51759111, 47762921, 55708102, 63656365, 55339303, 57390230, 67830379, 61805032, 26091326, 54802759], "Natural Language processing": [252008, 18784729, 42799166, 57932194, 18863997, 13805160, 27857167, 4561188, 43771647, 61603971, 1661566, 43561218, 1936537, 11147298, 27837170, 6650456, 563439, 5561, 10235, 360030, 64695824, 32472154, 2891758, 21173, 56142183, 21652, 40573, 14003441, 60360004, 20892159, 32707853, 67147, 98778, 37764426, 301999, 53358397, 36323189, 62026514], "World Cup 2022": [42931572, 27007503, 60637832, 29868391, 64467696, 4743361, 60410401, 61269058, 62528055, 10822574, 36581929, 45271353, 67608822, 27226732, 61715824, 57240806, 2150801, 64999924, 16383, 57918704, 66040086, 66040084, 3556431, 17742072, 51765484, 11370, 64999764, 57918711, 1248592], "Dolly the sheep": [9146, 39379960, 48188481, 9649607, 192685, 1751707, 16285933, 45485344, 8716, 52793670, 1731036, 168927, 2372209, 1140293, 2828101, 56398129, 42555506, 932553, 7932132, 14020881, 915258, 6910, 14094, 1567101, 1321047, 1631732, 17842616, 913362, 8394105, 63031051, 6832430, 2082914, 38889846, 1962277, 17158563, 1632972, 1857574, 66603787, 53431353, 12054042, 383180], "Ciggarets": [5801264, 2236126, 30942, 3015678, 55822753, 56132631, 10833234, 55836725, 280437, 11996885, 2672131, 4576717, 43345713, 46734540, 50164035, 4870997, 489585, 25913130, 2418612, 11938696, 56178521, 2761281, 3915251, 9254970, 1556887, 655861, 52958915, 8655214, 20587357, 73298, 56107088, 63616836, 5892113, 17596651, 56000054, 38327, 6003061, 14501317, 2199688, 13834142, 35077599, 1910732, 312963, 2627188, 54258598, 2536648, 1287604, 32695480], "What is the best place to live in?": [53487, 22989, 37321573, 80735, 3138, 645042, 33018516, 2694428, 124779, 1242998, 3928523, 844, 16760693, 58586, 60333700, 8522, 53837, 3708, 200427, 13602714, 42881894, 47744894, 24534207, 99648, 1664254, 28139692, 309890, 199292, 31885991, 311130, 5391, 5201333, 47789, 5407, 19159283, 109780, 1649321, 784781, 32950054, 48461477, 26976, 23189729, 37325161, 22912415, 1998, 3367760, 34361], "Elon musk": [52247588, 43407192, 8046414, 48778030, 31406060, 51237650, 21821257, 36971117, 65175052, 2614738, 66405413, 55947330, 5533631, 65212863, 53215263, 832774, 909036, 4335905, 50399439, 51714024, 803102, 45111627, 53615490, 9988187, 48795986], "How do you breed flowers?": [30876044, 4576465, 16128216, 33336442, 26537, 57374888, 63180590, 68213121, 893280, 31552410, 3288269, 224785, 1183979, 1104639, 1028614, 19049100, 277231, 42680256, 1392524, 407234, 18967, 55819873, 63539530, 57141131, 56170677, 233609, 3514423, 200646, 39683, 66556, 6614349, 41244, 5902061, 167906, 35646178, 63484108, 13799261, 18691124, 4226137, 1390689, 2327234, 33131935, 630109, 76143, 18952271, 1071613, 971961]}

In [29]:
import itertools
slices_dict = dict(itertools.islice(train_dict.items(),20))
docs = []
for q,d in slices_dict.items():
  docs+=d

#**Inverted Index for Documents Body**

In [30]:
id_body_pairs = parquetFile.filter(parquetFile["id"].isin(docs)).select("text", "id").rdd
# id_body_pairs = parquetFile.select("text", "id").rdd


In [31]:
def calculate_TT(pls):
  sum = 0
  for tup in pls:
    sum += tup[1]
  return sum


# word counts map
word_counts_body = id_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
trashhold = 1  # 50?!
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>=trashhold)


token_tt = postings_filtered_body.map(lambda x: (x[0],calculate_TT(x[1])))

term_total_dict = token_tt.collectAsMap()


w2df_body = calculate_df(postings_filtered_body)
w2df_body_dict = w2df_body.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered_body, file_name="body").collect()


In [32]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/body_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Create the Body Inverted Index

In [34]:
def tokenize(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return tokens


body_docs_tokenized = id_body_pairs.map(lambda x:(x[1], len(tokenize(x[0]))))



DL_dict = body_docs_tokenized.collectAsMap()


# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_body_dict
inverted_body.term_total = term_total_dict

inverted_body.DL = DL_dict

inverted_body.N = len(DL_dict)

total = 0
vals = list(DL_dict.values())
for ele in vals:
    total = total + ele

inverted_body.AVGDL = total / len(DL_dict)

# write the global stats out
inverted_body.write_index('.', 'Body_Index')
# upload to gs
index_src = "Body_Index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/body_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://Body_Index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      


#**Inverted Index for Documents Title**

In [35]:
doc_title = parquetFile.filter(parquetFile["id"].isin(docs)).select("title", "id").rdd
# doc_title = parquetFile.select("title", "id").rdd


In [37]:
count_title = doc_title.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = count_title.groupByKey().mapValues(reduce_word_counts)

w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()

title_docs_tokenized = doc_title.map(lambda x:(x[1], len(tokenize(x[0]))))

DL_dict_title = title_docs_tokenized.collectAsMap()

In [39]:
_ = partition_postings_and_write(postings_title, file_name="title").collect()

In [40]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/title_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [41]:
# Create inverted index instance
inverted_index_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_index_title.posing_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_index_title.df = w2df_title_dict


inverted_index_title.DL = DL_dict_title



inverted_index_title.N = len(DL_dict_title)

total = 0
vals = list(DL_dict_title.values())
for ele in vals:
    total = total + ele

inverted_index_title.AVGDL = total / len(DL_dict_title)



# write the global stats out
inverted_index_title.write_index('.', 'Title_Index')

# upload to gs
index_src = "Title_Index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/title_index/{index_src}'
!gsutil cp $index_src $index_dst


Copying file://Title_Index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      


#**Inverted Index for Documents Anchors**

In [64]:
# doc_anchors = parquetFile.filter(parquetFile["id"].isin(docs)).select("anchor_text", "id").rdd

doc_anchors = parquetFile.filter(parquetFile["id"].isin(docs)).select("anchor_text").rdd


In [59]:
def to_doc_anchor_pairs(entry):
    pairs = []
    for doc_id, anchor_text in entry[0]:
        pairs.append((doc_id, anchor_text))
    return pairs

all_anchors_to_id = doc_anchors.flatMap(to_doc_anchor_pairs).groupByKey().mapValues(list).map(lambda x: (" ".join(x[1]), x[0]))



word_count_anchors = all_anchors_to_id.flatMap(lambda x: word_count(x[0], x[1]))
postings_anchors = word_count_anchors.groupByKey().mapValues(reduce_word_counts)

w2df_anchors = calculate_df(postings_anchors)
w2df_anchors_dict = w2df_anchors.collectAsMap()

In [48]:
_ = partition_postings_and_write(postings_anchors, file_name="anchor").collect()

In [50]:
super_posting_locs_anchors = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp/anchors_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_anchors[k].extend(v)

In [53]:
# Create inverted index instance
inverted_index_anchors = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_index_anchors.posting_locs = super_posting_locs_anchors
# Add the token - df dictionary to the inverted index
inverted_index_anchors.df = w2df_anchors_dict
# write the global stats out
inverted_index_anchors.write_index('.', 'Anchors_Index')

# upload to gs
index_src = "Anchors_Index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/anchor_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://Anchors_Index.pkl [Content-Type=application/octet-stream]...
/ [1 files][619.4 KiB/619.4 KiB]                                                
Operation completed over 1 objects/619.4 KiB.                                    


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [61]:
# Put your `generate_graph` function here
def generate_graph(pages):

  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE


  edges = pages.flatMap(lambda entry: set([(entry[0],link) for link,title in entry[1]]))
  v = {(element,) for tupl in edges.toLocalIterator() for element in tupl}
  vertices = sc.parallelize(v)


  return edges, vertices

#**Page Rank Calc**

In [66]:
# pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# doc_acnchors = parquetFile.filter(parquetFile["id"].isin(docs)).select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(doc_acnchors)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
# pr.show()



+--------+------------------+
|      id|          pagerank|
+--------+------------------+
|   15271| 2.951077634298029|
|13873200|2.8927633104876236|
|  840273|2.2150085782523563|
| 3434750| 2.129785015526203|
|   30680|2.1188043686723943|
|  205878| 1.953730684950665|
| 4166365| 1.937578780952356|
|19619306|1.8824261665396962|
|  400199|1.8324890460500094|
|   14533|1.8109777498875748|
|    8460|1.7812268669908118|
|    8389| 1.748907509319908|
| 4059023|1.7025993646132225|
|  269496|1.6790516259920998|
|  442684|1.6724645274258414|
|  273700|1.6389329797536096|
|22101487|1.5991774644963597|
|     856|1.5929955244387297|
| 7515964|  1.58100957032416|
|   15881|1.5670002738277045|
+--------+------------------+
only showing top 20 rows



In [84]:
pr_dict = pr.toPandas().set_index('id').T.to_dict('list')

save_index_to_pickle("page_rank", pr_dict)

#**Page Views Calc**

In [ ]:
# # Paths
# # Using user page views (as opposed to spiders and automated traffic) for the 
# # month of August 2021
from pathlib import Path
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# # Download the file (2.3GB) 
!wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly 
# # total number of page views (5). Then, remove lines with article id or page 
# # view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# # Create a Counter (dictionary) that sums up the pages views for the same 
# # article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# # write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# # # read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

--2023-01-04 11:29:31--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.70MB/s    in 8m 22s  

2023-01-04 11:37:53 (4.76 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [82]:
save_index_to_pickle("page_views",wid2pv)

#**Create Dictionary Of Doc_id to Title**

In [68]:
# doc_acnchors = parquetFile.select("id", "title").rdd.collect()
# doc_acnchors = parquetFile.filter(parquetFile["id"].isin(docs)).select("id", "anchor_text").rdd

id_title_dict = dict(doc_title.collect())

with open("id_title_dict.pkl", "wb") as f:
    pickle.dump(id_title_dict, f)

bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob("postings_gcp/id_title_dict.pkl")
blob_posting_locs.upload_from_filename("id_title_dict.pkl")